#Import the Libraries

In [1]:
#import data manipulation packages
import os
import random
import shutil
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#import deep learning tools 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

#Download File Zip Dataset

In [2]:
#!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1k5pNW5OhEXkDcjaKmuabF8WXdd9uX5zd' -O 'EYEAM_Image_Data.zip'
#!wget -O file https://googledrive.com/host/1k5pNW5OhEXkDcjaKmuabF8WXdd9uX5zd
!gdown --id 1k5pNW5OhEXkDcjaKmuabF8WXdd9uX5zd -O /tmp/EYEAM_Image_Data.zip
#!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1AnsoyBESGSYzRvbMQh5-FWJdgtTo_gOj' -O 'Kijij Listings - edited.xlsx'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1k5pNW5OhEXkDcjaKmuabF8WXdd9uX5zd
To: /tmp/EYEAM_Image_Data.zip
100% 369M/369M [00:01<00:00, 199MB/s]


#Extract File Zip Dataset

In [3]:
local_zip = '/tmp/EYEAM_Image_Data.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [4]:
source_path = '/tmp/EYEAM Image Data'

source_path_binturong = os.path.join(source_path, "Binturong")
source_path_koala = os.path.join(source_path, "Koala")
source_path_sanfords_brown_lemur = os.path.join(source_path, "Sanford_s Brown Lemur")
source_path_siau_island_tarsier = os.path.join(source_path, "Siau Island Tarsier")
source_path_walrus = os.path.join(source_path, "Walrus")


# os.listdir returns a list containing all files under the given path
print(f"There are {len(os.listdir(source_path_binturong))} images of Binturong.")
print(f"There are {len(os.listdir(source_path_koala))} images of Koala.")
print(f"There are {len(os.listdir(source_path_sanfords_brown_lemur))} images of Sanford's Brown Lemur.")
print(f"There are {len(os.listdir(source_path_siau_island_tarsier))} images of Siau Island Tarsier.")
print(f"There are {len(os.listdir(source_path_walrus))} images of Walrus.")

There are 55 images of Binturong.
There are 50 images of Koala.
There are 50 images of Sanford's Brown Lemur.
There are 50 images of Siau Island Tarsier.
There are 50 images of Walrus.


In [5]:
# Define root directory
root_dir = '/tmp/endangered-animals'

# Empty directory to prevent FileExistsError is the function is run several times
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

# GRADED FUNCTION: create_train_test_dirs
def create_train_test_dirs(root_path):
  ### START CODE HERE
  training_dir = os.path.join(root_path, "training")
  testing_dir  = os.path.join(root_path, "testing")

  training_binturongs_dir = os.path.join(training_dir, "binturongs")
  training_koalas_dir = os.path.join(training_dir, "koalas")
  training_sanfords_brown_lemurs_dir = os.path.join(training_dir, "sanford_s_brown_lemurs")
  training_siau_island_tarsiers_dir = os.path.join(training_dir, "siau_island_tarsiers")
  training_walruses_dir = os.path.join(training_dir, "walruses")
  
  testing_binturongs_dir = os.path.join(testing_dir, "binturongs")
  testing_koalas_dir = os.path.join(testing_dir, "koalas")
  testing_sanfords_brown_lemurs_dir = os.path.join(testing_dir, "sanford_s_brown_lemurs")
  testing_siau_island_tarsiers_dir = os.path.join(testing_dir, "siau_island_tarsiers")
  testing_walruses_dir = os.path.join(testing_dir, "walruses")

  os.makedirs(training_binturongs_dir, exist_ok = True)
  os.makedirs(training_koalas_dir, exist_ok = True)
  os.makedirs(training_sanfords_brown_lemurs_dir, exist_ok = True)
  os.makedirs(training_siau_island_tarsiers_dir, exist_ok = True)
  os.makedirs(training_walruses_dir, exist_ok = True)
  
  os.makedirs(testing_binturongs_dir, exist_ok = True)
  os.makedirs(testing_koalas_dir, exist_ok = True)
  os.makedirs(testing_sanfords_brown_lemurs_dir, exist_ok = True)
  os.makedirs(testing_siau_island_tarsiers_dir, exist_ok = True)
  os.makedirs(testing_walruses_dir, exist_ok = True)
  # HINT:
  # Use os.makedirs to create your directories with intermediate subdirectories
  # Don't hardcode the paths. Use os.path.join to append the new directories to the root_path parameter

  pass
  
  ### END CODE HERE

  
try:
  create_train_test_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [6]:
# Test your create_train_test_dirs function
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

/tmp/endangered-animals/testing
/tmp/endangered-animals/training
/tmp/endangered-animals/testing/binturongs
/tmp/endangered-animals/testing/sanford_s_brown_lemurs
/tmp/endangered-animals/testing/walruses
/tmp/endangered-animals/testing/siau_island_tarsiers
/tmp/endangered-animals/testing/koalas
/tmp/endangered-animals/training/binturongs
/tmp/endangered-animals/training/sanford_s_brown_lemurs
/tmp/endangered-animals/training/walruses
/tmp/endangered-animals/training/siau_island_tarsiers
/tmp/endangered-animals/training/koalas


In [7]:
from shutil import copyfile
# GRADED FUNCTION: split_data
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  files = []
  for file_name in os.listdir(SOURCE):
    the_file = SOURCE + file_name
    if os.path.getsize(the_file) > 0:
      files.append(file_name)
    else:
      print(file_name + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[:testing_length]

  ### START CODE HERE
  for file_name in training_set:
    file_this = SOURCE + file_name
    dest     = TRAINING + file_name
    copyfile(file_this, dest)

  for file_name in testing_set:
      file_this = SOURCE + file_name
      dest     = TESTING + file_name
      copyfile(file_this, dest)
  pass


  ### END CODE HERE

In [18]:
# Test your split_data function

# Define paths
BINTURONGS_SOURCE_DIR = "/tmp/EYEAM Image Data/Binturong/"
KOALAS_SOURCE_DIR = "/tmp/EYEAM Image Data/Koala/"
SANFORD_S_BROWN_LEMURS_SOURCE_DIR = "/tmp/EYEAM Image Data/Sanford_s Brown Lemur/"
SIAU_ISLAND_TARSIERS_SOURCE_DIR = "/tmp/EYEAM Image Data/Siau Island Tarsier/"
WALRUSES_SOURCE_DIR = "/tmp/EYEAM Image Data/Walrus/"

TRAINING_DIR = "/tmp/endangered-animals/training/"
TESTING_DIR = "/tmp/endangered-animals/testing/"

TRAINING_BINTURONGS_DIR = os.path.join(TRAINING_DIR, "binturongs/")
TESTING_BINTURONGS_DIR = os.path.join(TESTING_DIR, "binturongs/")

TRAINING_KOALAS_DIR = os.path.join(TRAINING_DIR, "koalas/")
TESTING_KOALAS_DIR = os.path.join(TESTING_DIR, "koalas/")

TRAINING_SANFORD_S_BROWN_LEMURS_DIR = os.path.join(TRAINING_DIR, "sanford_s_brown_lemurs/")
TESTING_SANFORD_S_BROWN_LEMURS_DIR = os.path.join(TESTING_DIR, "sanford_s_brown_lemurs/")

TRAINING_SIAU_ISLAND_TARSIERS_DIR = os.path.join(TRAINING_DIR, "siau_island_tarsiers/")
TESTING_SIAU_ISLAND_TARSIERS_DIR = os.path.join(TESTING_DIR, "siau_island_tarsiers/")

TRAINING_WALRUSES_DIR = os.path.join(TRAINING_DIR, "walruses/")
TESTING_WALRUSES_DIR = os.path.join(TESTING_DIR, "walruses/")

# Empty directories in case you run this cell multiple times
if len(os.listdir(TRAINING_BINTURONGS_DIR)) > 0:
  for file in os.scandir(TRAINING_BINTURONGS_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_KOALAS_DIR)) > 0:
  for file in os.scandir(TRAINING_KOALAS_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_SANFORD_S_BROWN_LEMURS_DIR)) > 0:
  for file in os.scandir(TRAINING_SANFORD_S_BROWN_LEMURS_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_SIAU_ISLAND_TARSIERS_DIR)) > 0:
  for file in os.scandir(TRAINING_SIAU_ISLAND_TARSIERS_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_WALRUSES_DIR)) > 0:
  for file in os.scandir(TRAINING_WALRUSES_DIR):
    os.remove(file.path)

if len(os.listdir(TESTING_BINTURONGS_DIR)) > 0:
  for file in os.scandir(TESTING_BINTURONGS_DIR):
    os.remove(file.path)
if len(os.listdir(TESTING_KOALAS_DIR)) > 0:
  for file in os.scandir(TESTING_KOALAS_DIR):
    os.remove(file.path)
if len(os.listdir(TESTING_SANFORD_S_BROWN_LEMURS_DIR)) > 0:
  for file in os.scandir(TESTING_SANFORD_S_BROWN_LEMURS_DIR):
    os.remove(file.path)
if len(os.listdir(TESTING_SIAU_ISLAND_TARSIERS_DIR)) > 0:
  for file in os.scandir(TESTING_SIAU_ISLAND_TARSIERS_DIR):
    os.remove(file.path)
if len(os.listdir(TESTING_WALRUSES_DIR)) > 0:
  for file in os.scandir(TESTING_WALRUSES_DIR):
    os.remove(file.path)

# Define proportion of images used for training
split_size = .9

# Run the function
# NOTE: Messages about zero length images should be printed out
split_data(BINTURONGS_SOURCE_DIR, TRAINING_BINTURONGS_DIR, TESTING_BINTURONGS_DIR, split_size)
split_data(KOALAS_SOURCE_DIR, TRAINING_KOALAS_DIR, TESTING_KOALAS_DIR, split_size)
split_data(SANFORD_S_BROWN_LEMURS_SOURCE_DIR, TRAINING_SANFORD_S_BROWN_LEMURS_DIR, TESTING_SANFORD_S_BROWN_LEMURS_DIR, split_size)
split_data(SIAU_ISLAND_TARSIERS_SOURCE_DIR, TRAINING_SIAU_ISLAND_TARSIERS_DIR, TESTING_SIAU_ISLAND_TARSIERS_DIR, split_size)
split_data(WALRUSES_SOURCE_DIR, TRAINING_WALRUSES_DIR, TESTING_WALRUSES_DIR, split_size)

# Check that the number of images matches the expected output
print(f"There are {len(os.listdir(TRAINING_BINTURONGS_DIR))} images of binturongs for training")
print(f"There are {len(os.listdir(TESTING_BINTURONGS_DIR))} images of binturongs for testing")

print(f"\n\nThere are {len(os.listdir(TRAINING_KOALAS_DIR))} images of koalas for training")
print(f"There are {len(os.listdir(TESTING_KOALAS_DIR))} images of koalas for testing")

print(f"\n\nThere are {len(os.listdir(TRAINING_SANFORD_S_BROWN_LEMURS_DIR))} images of sanford's brown_lemurs for training")
print(f"There are {len(os.listdir(TESTING_SANFORD_S_BROWN_LEMURS_DIR))} images of sanford's brown_lemurs for testing")

print(f"\n\nThere are {len(os.listdir(TRAINING_SIAU_ISLAND_TARSIERS_DIR))} images of siau island tarsiers for training")
print(f"There are {len(os.listdir(TESTING_SIAU_ISLAND_TARSIERS_DIR))} images of siau island tarsiers for testing")

print(f"\n\nThere are {len(os.listdir(TRAINING_WALRUSES_DIR))} images of walruses for training")
print(f"There are {len(os.listdir(TESTING_WALRUSES_DIR))} images of walruses for testing")

There are 38 images of binturongs for training
There are 17 images of binturongs for testing


There are 35 images of koalas for training
There are 15 images of koalas for testing


There are 35 images of sanford's brown_lemurs for training
There are 15 images of sanford's brown_lemurs for testing


There are 35 images of siau island tarsiers for training
There are 15 images of siau island tarsiers for testing


There are 35 images of walruses for training
There are 15 images of walruses for testing


In [19]:
# GRADED FUNCTION: train_val_generators
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  ### START CODE HERE

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  train_datagen = ImageDataGenerator(rescale = 1.0/255.)

  # Pass in the appropiate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=20,
                                                      class_mode="binary",
                                                      target_size=(150, 150))

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  validation_datagen = ImageDataGenerator(rescale = 1.0/255.)

  # Pass in the appropiate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                      batch_size=20,
                                                      class_mode="binary",
                                                      target_size=(150, 150))
  ### END CODE HERE
  return train_generator, validation_generator

In [20]:
# Test your generators
train_generator, validation_generator = train_val_generators(TRAINING_DIR, TESTING_DIR)

Found 178 images belonging to 5 classes.
Found 77 images belonging to 5 classes.


In [21]:
# GRADED FUNCTION: create_model
def create_model():
  # DEFINE A KERAS MODEL TO CLASSIFY CATS V DOGS
  # USE AT LEAST 3 CONVOLUTION LAYERS

  ### START CODE HERE

  model = tf.keras.models.Sequential([
  # YOUR CODE HERE
      tf.keras.layers.Conv2D(32,(3,3), activation = 'relu', input_shape=(150,150,3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128,(3,3), activation = 'relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation = 'relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])


  
  model.compile(optimizer=Adam(learning_rate=0.001),
                loss='binary_crossentropy',
                metrics=['accuracy']) 
    
  ### END CODE HERE

  return model


In [26]:
# Get the untrained model
model = create_model()

# Train the model
# Note that this may take some time.
history = model.fit(train_generator,
                    steps_per_epoch = 40,
                    epochs=15,
                    validation_data=validation_generator,
                    validation_steps = 27,
                    verbose=1)

Epoch 1/15
40/40 [==============================] - 13s 290ms/step - loss: -526.3801 - accuracy: 0.1798 - val_loss: -2487.1135 - val_accuracy: 0.1948


In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.show()
print("")

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.show()